In [ ]:
import sys
import os
import cv2
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import sklearn
from utils import *
%matplotlib inline

In [ ]:
train_speed = np.loadtxt('./data/train.txt')
total_frame = train_speed.shape[0]
plt.plot(np.arange(0,len(train_speed)/20,1/20),train_speed)
plt.xlabel('time(s)')
plt.ylabel('speed(mph)')
plt.title('Speed Tracking')
plt.show()

In [ ]:
info = pd.read_csv('./data/info.csv')
info['avg_speed'] = info.label.rolling(window=2).mean()
curPaths= [None]
for i in range(total_frame-1):
    curPaths.append('./data/training_frames/' + str(i)+'.jpg')
info['curImgPath'] = curPaths
info = info.drop(info.index[0])
info = info.rename(columns={'path':'nxtImgPath'})
info = info.reset_index(drop=True)
info.head()

In [ ]:
shuffled = sklearn.utils.shuffle(info)
train_number =int(total_frame*0.7)
validation_number = int(total_frame*0.2)
test_number = int(total_frame*0.1)

train_info = shuffled.iloc[:train_number]
validation_info = shuffled.iloc[train_number:train_number+validation_number]
test_info = shuffled.iloc[train_number+validation_number:]
print('train_number: ' + str(len(train_info)))
print('validation number: ' + str(len(validation_info)))
print('test_number: ' + str(len(test_info)))

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
filepath = 'nvidia_2.h5'
earlyStopping = EarlyStopping(monitor='val_loss', 
                              patience=1, 
                              verbose=1, 
                              min_delta = 0.2,
                              mode='min',)
modelCheckpoint = ModelCheckpoint(filepath, 
                                  monitor = 'val_loss', 
                                  save_best_only = True, 
                                  mode = 'min', 
                                  verbose = 1,
                                 save_weights_only = True)
callbacks_list = [modelCheckpoint]
train_generator = batch_generator(train_info)
valid_generator = batch_generator(validation_info)


In [ ]:
model = nvidia_model()
history = model.fit_generator(
        train_generator, 
        steps_per_epoch = len(train_info)//16, 
        epochs = 15,
        callbacks = callbacks_list,
        verbose = 1,
        validation_data = valid_generator,
        validation_steps = len(validation_info)//16)

#print(history)